In [1]:
import pandas as pd
import re
import math as math
history = pd.read_csv("../data/raw/history_full_data.csv")

In [2]:
import re
def extract_year_price(row):
    string = {}
    price_string = row['cost_text'].replace(",","")
    price_year = re.findall(r'\d+[$]\d+', price_string)
    for each_year in price_year:
        year = re.search(r'\d+[$]', each_year).group(0).replace("$","")
        price = re.search(r'[$]\d+', each_year).group(0).replace("$","")
        if price and year:
            string[int(year)] = int(price)    
    return string

In [3]:
history.apply(lambda row:extract_year_price(row),axis=1)

0                               {2021: 500, 2020: 550}
1                                          {2021: 390}
2                                          {2021: 645}
3                                          {2020: 430}
4                               {2019: 420, 2017: 410}
                             ...                      
33857     {2022: 410, 2019: 310, 2017: 295, 2015: 260}
33858                                      {2022: 550}
33859                                   {2022: 820000}
33860    {2022: 480, 2016: 300, 2014: 1300, 2013: 300}
33861                                  {2022: 1395000}
Length: 33862, dtype: object

In [4]:
history["price_in_history"] = history.apply(lambda row:extract_year_price(row),axis=1)

In [5]:
# counting the # of rows
for year in range(2017,2022):
    count = 0
    for index in range(33862):
        if year in history.iloc[index]['price_in_history']:
            count += 1
    print(f'{year}:{count}')

2017:4469
2018:4911
2019:5460
2020:5747
2021:6903


In [6]:
history['year'] = history.apply(lambda row:list(row['price_in_history'].items()),axis=1)

In [7]:
history = history.explode('year')

In [8]:
history = history[history['price_in_history']!={}]

In [9]:
history["years"]= history.apply(lambda row:row['year'][0],axis=1)
history["price"]= history.apply(lambda row:row['year'][1],axis=1)

In [10]:
history = history.drop(['Unnamed: 0', 'cost_text','price_in_history','year'], axis=1)

In [11]:
# overview about spliting the rent of each year
history['postcode'] = history['postcode'].astype('str')

# Data preprocessing 2017-2021

In [12]:
# getting the the table postcodes matches to SA2 region name
# The data scrape from https://www.matthewproctor.com/full_australian_postcodes_vic
import requests
from bs4 import BeautifulSoup
url = "https://www.matthewproctor.com/full_australian_postcodes_vic" #网址
data = requests.get(url).text  #获取数据
soup = BeautifulSoup(data, 'html.parser') #获取数据中想要的东西
table = soup.find("table", {"class": "table table-striped"}) #获取其中的表格
df = pd.DataFrame(columns=['postcodes', 'SA2_names'])

# Collecting Ddata
for row in table.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        postcode = columns[1].text.strip()
        SA2_name = columns[19].text.strip()
        df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
sa2_dataframe = df.iloc[1:]
def add_sa2(row,sa2_dataframe):
    postcode = row['postcode']
    selected_postcode = sa2_dataframe.loc[sa2_dataframe['postcodes'] == postcode]
    sa2 = [(selected_postcode.iloc[i])["SA2_names"] for i in range(len(selected_postcode))]
    sa2=list(set(sa2))
    return sa2
history['sa2_region'] = history.apply(lambda row: add_sa2(row,sa2_dataframe), axis=1)

/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_4511/1042629386.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_4511/1042629386.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_4511/1042629386.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcodes': postcode,'SA2_names': SA2_name}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_4511/1042629386.py:19: FutureWarning: The frame

In [13]:
SA2_centroids = pd.read_csv('../data/raw/mb_2016_centroids.csv')
all_value = []
SA2_vic = SA2_centroids[SA2_centroids['STE_NAME16']=='Victoria']
SA2_vic = SA2_vic[['SA2_MAIN16','SA2_NAME16','xcoord','ycoord']]
SA2_vic.head()
SA2_vic_xcoord = SA2_vic.groupby('SA2_NAME16')["xcoord"].mean()
SA2_vic_ycoord = SA2_vic.groupby('SA2_NAME16')["ycoord"].mean()
#现在拿到了所有SA2的中心区域
SA2_vic_xy = pd.merge(SA2_vic_xcoord,SA2_vic_ycoord,left_on = 'SA2_NAME16',right_on = 'SA2_NAME16')
#groupy the SA2_NAME16 and take the average of xcoord and ycoord
SA2_vic_xy=dict(SA2_vic_xy.transpose())
all_co = []
for value in history['sa2_region']:
    ind_co=[] #每一行的结果
    for region in value:
        #print(region)
        if region in SA2_vic_xy.keys():
            ind_co.append(SA2_vic_xy[region])
    all_co.append(ind_co)
history['sa2_coor'] = all_co

In [14]:
#function to calculate the distance 
def getDistance(latA, lonA, latB, lonB):
    ra = 6378140  # 赤道半徑
    rb = 6356755  # 極半徑
    flatten = (ra - rb) / ra  # Partial rate of the earth
    # change angle to radians
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)

    pA = math.atan(rb / ra * math.tan(radLatA))
    pB = math.atan(rb / ra * math.tan(radLatB))
    x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(radLonA - radLonB))
    c1 = (math.sin(x) - x) * (math.sin(pA) + math.sin(pB)) ** 2 / math.cos(x / 2) ** 2
    c2 = (math.sin(x) + x) * (math.sin(pA) - math.sin(pB)) ** 2 / math.sin(x / 2) ** 2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (x + dr)
    distance = round(distance / 1000, 4)
    return distance

In [15]:
# 找出离房子最近的sa2
def close_sa2(row): 
    all_distance = [getDistance(row['latitude'],row['longitude'],x[1],x[0]) for x in row.sa2_coor]
    min_distance = min(all_distance,default = 0)
    return row['sa2_region'][all_distance.index(min_distance)]   
history['closest_sa2'] = history.apply(lambda row:close_sa2(row),axis=1)

# Merge the 2017-2021 population and income into the dataset

In [16]:
#merge the income By SA2
#extract the income of year 2017-2021
all_income = pd.read_csv('../data/raw/Income_Prediction.csv')

In [17]:
all_income = all_income.drop(columns=['Unnamed: 0'])

In [18]:
# Manually filling the income of Melbourne 
# https://www.averagesalarysurvey.com/melbourne-australia
data = {'SA2 code': [206041122],
        'SA2 name': ['Melbourne'],
        '2010': [64792],
        '2011':[69316],
        '2012':[71532],
        '2013':[72800],
        '2014':[74827],
        '2015':[76189],
        '2017Pred':[87360],
        '2018Pred':[91230],
        '2019Pred':[99820],
        '2020Pred':[116530],
        '2021Pred':[120000],
        '2022Pred':[126197],
        '2023Pred':[129324],
        '2024Pred':[134234],
        '2026Pred':[136249],
        '2027Pred':[140124]}  
# Create DataFrame  
df = pd.DataFrame(data)  
all_income = all_income.append(df)

/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_4511/4119110117.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_income = all_income.append(df)


In [19]:
all_income_2017 = all_income[['2017Pred','SA2 name']]
all_income_2018 = all_income[['2018Pred','SA2 name']]
all_income_2019 = all_income[['2019Pred','SA2 name']]
all_income_2020 = all_income[['2020Pred','SA2 name']]
all_income_2021 = all_income[['2021Pred','SA2 name']]
history_17 = history[history['years'] == 2017]
history_18 = history[history['years'] == 2018]
history_19 = history[history['years'] == 2019]
history_20 = history[history['years'] == 2020]
history_21 = history[history['years'] == 2021]
history_17 = pd.merge(history_17,all_income_2017,how='inner',left_on = 'closest_sa2',right_on = 'SA2 name')
history_18 = pd.merge(history_18,all_income_2018,how='inner',left_on = 'closest_sa2',right_on = 'SA2 name')
history_19 = pd.merge(history_19,all_income_2019,how='inner',left_on = 'closest_sa2',right_on = 'SA2 name')
history_20 = pd.merge(history_20,all_income_2020,how='inner',left_on = 'closest_sa2',right_on = 'SA2 name')
history_21 = pd.merge(history_21,all_income_2021,how='inner',left_on = 'closest_sa2',right_on = 'SA2 name')

In [20]:
df.rename({"Old_Column_Name_1":"New_Column_Name_2","Old_Column_Name_2":"New_Column_Name_2"},axis=1, inplace=True)

In [21]:
history_17.rename({"2017Pred":'Income'},axis=1, inplace=True)
history_18.rename({"2018Pred":'Income'},axis=1, inplace=True)
history_19.rename({"2019Pred":'Income'},axis=1, inplace=True)
history_20.rename({"2020Pred":'Income'},axis=1, inplace=True)
history_21.rename({"2021Pred":'Income'},axis=1, inplace=True)


In [22]:
all_population = pd.read_csv('../data/raw/population_final_predict.csv')
all_population.head()

,Unnamed: 0,SA2 code,2017 Population,2018 Population,2019 Population,2020 Population,2021 Population,2022 Population,2026 Population,2031 Population,2036 Population,SA2 name
0,0,201011001,12639,13520,14408,15472,16823,16623.337207,21796.4,26956.4,32116.4,Alfredton
1,1,201011002,12261,12237,12311,12183,12076,12366.053200,11916.8,11704.8,11492.8,Ballarat
2,2,201011005,7407,7413,7452,7369,7232,7749.168968,7098.8,6901.8,6704.8,Buninyong
3,3,201011006,7617,8174,8876,9736,10640,10722.789138,14334.2,18138.2,21942.2,Delacombe
4,4,201011007,4002,4040,4109,4148,4213,4176.433099,4473.4,4738.4,5003.4,Smythes Creek


In [23]:
all_population = all_population.drop(columns=['Unnamed: 0'])

In [24]:
melbourne_population = all_population[all_population['SA2 name'].str.contains('Melbourne CBD')].sum(axis=0)
melbourne_population['SA2 name'] = 'Melbourne'

In [25]:
all_population = all_population.append(melbourne_population,ignore_index=True)

/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_4511/2658661369.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_population = all_population.append(melbourne_population,ignore_index=True)


In [26]:
all_population.drop(all_population.index[135:138], inplace=True)

In [27]:
#merge the population
all_population_2017 = all_population[['2017 Population','SA2 name']]
all_population_2018 = all_population[['2018 Population','SA2 name']]
all_population_2019 = all_population[['2019 Population','SA2 name']]
all_population_2020 = all_population[['2020 Population','SA2 name']]
all_population_2021 = all_population[['2021 Population','SA2 name']]
history_17 = pd.merge(history_17,all_population_2017,how='inner',left_on = 'closest_sa2',right_on = 'SA2 name')
history_18 = pd.merge(history_18,all_population_2018,how='inner',left_on = 'closest_sa2',right_on = 'SA2 name')
history_19 = pd.merge(history_19,all_population_2019,how='inner',left_on = 'closest_sa2',right_on = 'SA2 name')
history_20 = pd.merge(history_20,all_population_2020,how='inner',left_on = 'closest_sa2',right_on = 'SA2 name')
history_21 = pd.merge(history_21,all_population_2021,how='inner',left_on = 'closest_sa2',right_on = 'SA2 name')

In [28]:
history_17.rename({"2017 Population":'Population'},axis=1, inplace=True)
history_18.rename({"2018 Population":'Population'},axis=1, inplace=True)
history_19.rename({"2019 Population":'Population'},axis=1, inplace=True)
history_20.rename({"2020 Population":'Population'},axis=1, inplace=True)
history_21.rename({"2021 Population":'Population'},axis=1, inplace=True)

In [29]:
history_all = pd.concat([history_17, history_18,history_19,history_20,history_21],axis = 0)

# Data preprocessing


In [30]:
#take the number of bedroom bathroom and car spaces 
import numpy as np
def data_preprocessing(dataset):
    bed_num=[]
    bath_num=[]
    park_num=[]
    #seperate the bedroom, bathroom and parking area
    for value in dataset['bed']:
        bed_num.append(re.findall(r'Bed : (\d+)', str(value))) #找到bed前面的数字
    for value in dataset['bath']:
        bath_num.append(re.findall(r'Bath : (\d+)', str(value)))
    for value in dataset['car']:
        park_num.append(re.findall(r'Car : (\d+)', str(value)))
    dataset['bed'] = bed_num
    dataset['bath'] = bath_num
    dataset['car'] = park_num
data_preprocessing(history_all)
def extract_bed(row):
    if row['bed'] == []:
        return None
    return row['bed'][0]
def extract_bath(row):
    if row['bath'] == []:
        return None
    return row['bath'][0]
def extract_car(row):
    if row['car'] == []:
        return None
    return row['car'][0]
history_all['bed'] = history_all.apply(lambda row:extract_bed(row),axis=1)
history_all['bath'] = history_all.apply(lambda row:extract_bath(row),axis=1)
history_all['car'] = history_all.apply(lambda row:extract_car(row),axis=1)
history_all = history_all.drop(labels=['SA2 name_x','SA2 name_y'], axis=1)
coordinates = []
for i in range(len(history_all)):
    coordinates.append((history_all[i:i+1]['latitude'],history_all[i:i+1]['longitude']))
history_all['coordinates'] = coordinates
history_all.rename({"price":'week_rent'},axis=1, inplace=True)
history_all = history_all[['name','years','property_type','coordinates','postcode','sa2_coor','closest_sa2','bed','bath','car','Income','Population','week_rent']]

In [31]:
history_all.head()

,name,years,property_type,coordinates,postcode,sa2_coor,closest_sa2,bed,bath,car,Income,Population,week_rent
0,"506/1 BOUVERIE STREET, CARLTON SOUTH",2017,Category : Unit/apmt,"([-37.8060974], [144.9612201])",3000,"[[144.9627648781132, -37.813376920301884]]",Melbourne,1,1,None,87360.0,44599,410
1,"911/551 SWANSTON STREET, CARLTON SOUTH",2017,Category : Unit/apmt,"([-37.8057236], [144.9624931])",3000,"[[144.9627648781132, -37.813376920301884]]",Melbourne,2,2,None,87360.0,44599,600
2,"UNIT 12/109 LYGON STREET, CARLTON SOUTH",2017,Category : Unit/apmt,"([-37.8045102], [144.9660078])",3000,"[[144.9627648781132, -37.813376920301884]]",Melbourne,1,1,None,87360.0,44599,250
3,"402 / 33 ROSE LANE, MELBOURNE",2017,NaN,"([-37.815235], [144.9541488])",3000,"[[144.9627648781132, -37.813376920301884]]",Melbourne,2,2,1,87360.0,44599,630
4,"911/199 WILLIAM STREET, MELBOURNE",2017,Category : Available Date,"([-37.8144054], [144.9575251])",3000,"[[144.9627648781132, -37.813376920301884]]",Melbourne,2,2,None,87360.0,44599,620


In [32]:
history_all.to_csv('../data/raw/2017_2021_house_data.csv')